In [1]:
import torch  # 导入torch包
import torch.nn as nn  # 导入神经网络模块库


class RMSNorm(torch.nn.Module):
    # 初始化函数
    def __init__(self, dim: int, eps: float = 1e-5):  # dim:特征维度；eps:一个很小的常数，为了防止除0
        super().__init__()  # 调用nn.Module的初始化函数
        self.eps = eps  # 将eps保存为实例属性，用于forward()
        # 创建可训练参数
        self.weight = nn.Parameter(torch.ones(dim))  # self.weight -> [dim, 1]
        '''
            - torch.ones(dim)：创建一个形状为(dim, )的全1一维张量
            - nn.Parameter():把输入的张量包装为Parameter
                -- 自动加入模型参数列表，model.parameters()会包含它
                -- 默认进行梯度计算，训练时会被优化器更新
                -- state_dict()会保存/加载它
        '''
        
        
    # RMSNorm的数学形式
    def _norm(self, x):  # x -> [..., dim]
        '''
        - x.pow(2)：对x做逐元素平方
        - .mean(-1, keepdim=True)：
            -- 在最后一个维度（索引-1）做均值，并保留维度（keepdim=True）
            -- 防止后面开方出现除0/数值爆炸
            -- torch.rsqrt(...)：等价于1/torch.sqrt(...)，
               函数实现上更高效、稳定
        '''
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
    
    
    # 前向传播
    def forward(self, x):
        '''
        - .float()：改变数值精度，默认是torch.float32
            -- 如果输入x原本是float16/float32（混合精度训练），
               归一化中mean/rsqrt在低精度下容易数值不稳定
        - .type_as(x)：把数值精度转回x的原本精度
            -- 归一化计算时float32数值稳定性更高，最终输出还是要和x保持一致
        
        '''
        return self.weight * self._norm(x.float()).type_as(x)  # 沿着最后维度做逐元素缩放
    # 代码测试
if __name__ == "__main__":
    dim = 512
    batch_size = 2
    seq_len = 10
    
    # 创建随机输入
    x = torch.randn(batch_size, seq_len, dim)
    print(f"x为:{x}")
    print(f"输入数据的形状：{x.shape}")
    print(f"输入数据的均值：{x.mean():.4f}，标准差：{x.std():.4f}")
    norm = RMSNorm(dim)
    x_norm = norm(x)
    print(f"归一化的形状：{x_norm.shape}")
    print(f"归一化的均值：{x_norm.mean():.4f},标准差：{x_norm.std():.4f}")


x为:tensor([[[-1.0812, -0.1391,  0.2784,  ...,  2.2696,  0.2764,  1.2323],
         [-0.0177,  1.5942, -1.9896,  ..., -0.5624,  1.5145, -1.0394],
         [-0.2164, -0.1913, -0.5676,  ..., -1.1281,  1.4255, -0.6159],
         ...,
         [-0.3048, -0.2480, -0.2798,  ...,  0.6624,  0.8622,  0.1906],
         [-1.3717, -1.7130,  1.0452,  ..., -0.6448,  1.0530, -0.2656],
         [ 0.5788,  0.3110,  0.2271,  ..., -0.7906,  1.5364,  0.4686]],

        [[-0.1672,  1.2301,  0.3381,  ...,  1.0543, -1.3415, -1.2647],
         [ 0.3526,  0.4110,  0.1657,  ...,  0.0686, -0.8454,  0.4099],
         [ 0.0595, -1.0151, -0.3275,  ...,  0.0814,  0.4463,  0.4587],
         ...,
         [-0.8346,  0.6087, -0.1771,  ..., -1.5772,  0.8458,  0.1620],
         [ 0.4974, -1.9349,  2.4758,  ...,  1.2780, -0.4603,  2.0665],
         [ 0.9666,  0.7984,  0.1407,  ...,  0.0025,  0.4696, -1.0767]]])
输入数据的形状：torch.Size([2, 10, 512])
输入数据的均值：0.0121，标准差：0.9946
归一化的形状：torch.Size([2, 10, 512])
归一化的均值：0.0123,标准差：1.00